<a href="https://colab.research.google.com/github/Zarat2209/Ex-Dat-Anal-assign-2/blob/master/binance_trade_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binance Trading Performance

This notebook retrieves your trade data from Binance to calculate the pnl from your trading.

The methodology for calculating PNL is:

1. Calculate the total amount of net base asset purchased (or sold) and the total amount of quote asset spent (or received)
2. Calculate the value of these changes based on current asset spot price
3. Calculate the value of the fees paid out in the trades

Note that this methodology is one way of trying to evaluate the benefit if your trading activity, i.e. doing something versus doing nothing.  This does not capture any changes in portfolio value due to general market movements that may result in the appreciation of the value of base assets and quote assets.

## Instructions

Step 1) Input your ***read only*** API keys

Step 2) Input trading pair and start date

Step 3) Select `Runtime` => `Run all`

## Notes

- `% gain and loss` is based on your current balance of base and quote asset.  This may not be a comprehensive figure if (1) you have made deposits/withdrawals within the period being analyzed, and (2) if you are trading multiple pairs with overlapping base and quote assets

## Comments / bugs / suggestions

Please email [carlo@hummingbot.io](mailto:carlol@hummingbot.io?subject=Colab:%20Performance%20Sheet).

## Input API keys

In [2]:
binance_api_key = "**ENTER API KEY**"
binance_api_secret = "**ENTER API SECRET**"

trading_pair = "COTIUSDT"
START_TIME = "2020-11-01 00:00:00"

# Setup

## Install dependencies

In [1]:
!pip install binance
!pip install python-binance
from binance.client import Client
import pandas as pd
import numpy as np
from requests import get
import json
import plotly.graph_objects as go
from IPython.core.display import display, HTML
from datetime import datetime

# Setup Binance connection
client = Client(binance_api_key, binance_api_secret)

# Set display
pd.options.display.float_format = '{:,.2f}'.format
start_dt = pd.to_datetime(START_TIME)

import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 3.1MB 7.1MB/s 
     |████████████████████████████████| 286kB 22.9MB/s 
     |████████████████████████████████| 1.5MB 25.6MB/s 
     |████████████████████████████████| 2.6MB 22.0MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 245kB 47.1MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 


NameError: ignored

# Run analysis

### Run performance analysis

In [ ]:
display(HTML(f"<h1>{trading_pair}: {start_dt} to {datetime.utcnow().replace(microsecond=0)}</h1>"))

trading_pair_info = client.get_symbol_info(trading_pair)
base_asset = trading_pair_info["baseAsset"]
quote_asset = trading_pair_info["quoteAsset"]

# Calculate current asset balances
df = pd.DataFrame(columns=["asset"], data=[base_asset, quote_asset])
df["price"] = df["asset"].apply(lambda x: get_usd_price(x))
df["balance"] = df["asset"].apply(lambda x: get_asset_balance(x))
df["usd_value"] = df["price"] * df["balance"]
df.set_index("asset", inplace=True, drop=True)

base_asset_price = df.at[base_asset, "price"]
quote_asset_price = df.at[quote_asset, "price"]

total_balance_usd = df['usd_value'].sum()
p_title(f"Current balance: ${total_balance_usd:,.2f}")
p_df(df)

# Fetch trades
df_trades = get_binance_trades(trading_pair, start_dt)

# Run performance analysis
calc_performance(df_trades)

,price,balance,usd_value
asset,,,
COTI,0.04,"5,478.61",213.01
USDT,1.00,231.25,231.25


first trade:          2020-11-01 00:07:33
last trade:           2020-11-20 05:57:58
total trades:         817
- buys:               438 / 53.6%
- sells:              379 / 46.4%
total base traded:    797,664
total quote traded:   27,315
approx. usd volume:  $31,013  (total base * current base price)


,Base (COTI),Quote (USDT),Total
Label,,,
Acquired,"398,103.80","13,703.34",-
Disposed,"399,560.20","13,611.29",-
Delta,"-1,456.40",92.05,-
Delta (usd),-56.62,92.05,35.43
Trading fees (usd value),-,-,-20.85
Net pnl (USD),-,-,14.57
% gain/loss,-,-,3.4%


,commission
commissionAsset,
BNB,0.72
COTI,5.59
USDT,0.37


In [ ]:
# Graph historical trades
p_title("Historical trades")
print(f"Amounts in {base_asset}")
graph_trades(df_trades)

Amounts in COTI


In [ ]:
p_title("All trades in period")
pd.set_option("display.max_rows", None)
df_trades